In [7]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc
import math
%matplotlib inline
import cv2
import time
import glob
from scipy.ndimage import gaussian_filter1d
from scipy import signal

img_list = []
filenamelist= []
for img in glob.glob("0525_images/CCD-3/*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(152,)

In [21]:
for iter_file in range(len(filenamelist)):
    
    #==============read in file==============================
    start = time.time()
    filename = filenamelist[iter_file]
    print(filename)
    image = np.array(cv2.imread(filename,0))
    backtorgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    
    #==============use canny to find the contour
#     image_blur = cv2.blur(image,(9,9))
#     canny = cv2.Canny(image_blur,5,20)
    ret, thresh1 = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY_INV)
    open_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,3))
    thresh1 = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, open_kernel)

    # ===========cut the ROI by the canny edge image================
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    approx_list = []
    index = []
    for i in range(np.array(contours).shape[0]):
        if (hierarchy[0][i][3] >= 0):
            if(cv2.contourArea(contours[i])>300000 and cv2.contourArea(contours[i])<500000):
                epsilon = 0.005# * cv2.arcLength(contours[i],True)
                index.append(i)
                approx = cv2.approxPolyDP(contours[i], epsilon, True)
                approx_list.append(approx)
    #==========================================find outer circle==============================================
    ret, thresh1 = cv2.threshold(image, 110, 255, cv2.THRESH_BINARY_INV)

    c,r = cv2.minEnclosingCircle(approx_list[0])
    r = int(r)
    cx = int(c[0])
    cy = int(c[1])
    # ===========mask the image to extract the inner circle================
    inner_contour_img = np.zeros_like(image)
    cv2.circle(inner_contour_img,(cx,cy),(r+7), (255,255,255),-1)
    #inner_contour_img = 1 - inner_contour_img
    oringmask = inner_contour_img
    image[oringmask == 0] = 255
    
    #============create 0.5 degree a line

    const = np.pi/180
    x = []
    y = []
    inner_x = []
    inner_y = []

    degree_delta = 0.5
    for degree in range(int(360//degree_delta)):

        degree_real = degree_delta*degree
        now_x = int((r+2)*np.sin(degree_real*const))+cx
        now_y = int((r+2)*np.cos(degree_real*const))+cy
        now_inner_x = int((r-30)*np.sin(degree_real*const))+cx
        now_inner_y = int((r-30)*np.cos(degree_real*const))+cy

        x.append(now_x)
        y.append(now_y)
        inner_x.append(now_inner_x)
        inner_y.append(now_inner_y)
        
        #============shot from center
        
    threshold_1phase = 30
    threshold_2phase = 10
    all_diff_list = []
    all_min_list = []
    degree_1phase_list = []
    for degree in range(int(360//degree_delta)):
        points_on_line = np.linspace((inner_x[degree], inner_y[degree]),(x[degree],y[degree]) ,200,dtype = int)
        pass_list = image[points_on_line[:,1],points_on_line[:,0]]
        
        #First use gaussian filter to make the curve smooth,and then find peaks and valley.
        pass_list = gaussian_filter1d(pass_list,4)
        peaks, _ = signal.find_peaks(pass_list, height=0, prominence = 10)
        valley, _ = signal.find_peaks(-pass_list, height=0, prominence = 10)
        peak_and_valley_list = np.concatenate((pass_list[peaks], pass_list[valley]), axis=0)
        
        #cacluate the difference of every peak-valley
        shift_peak_and_valley_list = np.concatenate((np.array([0]), peak_and_valley_list))
        peak_and_valley_list = np.concatenate((peak_and_valley_list,np.array([0])))
        diff_list = abs(peak_and_valley_list - shift_peak_and_valley_list)    
        diff_list = diff_list[1::2]
        
        if(len(peaks)==0 or len(valley)==0 ):#如果沒找到local min 和max
            cv2.line(backtorgb,(inner_x[degree],inner_y[degree]),(x[degree],y[degree]),255,2)
            cv2.circle(backtorgb,(x[degree],y[degree]),30, (0,255 ,255), 5)
            continue
            
        now_diff = max(diff_list)
        all_diff_list.append(now_diff)
        all_min_list.append(min(pass_list[valley]))
        
        if((min(pass_list[valley])>120 or now_diff<threshold_1phase)): #如果最小的值>120
            degree_1phase_list.append(degree)
        
    for i in range(len(degree_1phase_list)):#第一階段有問題的degree，第二階段看看是不是真的有問題
        degree = degree_1phase_list[i]
        now_value = all_min_list[degree]
        prev_value = all_min_list[degree-1]
        next_value = all_min_list[degree+1]
        
        if(abs(float(prev_value)-float(now_value))>threshold_2phase and abs(float(next_value) - float(now_value))>threshold_2phase):
            cv2.line(backtorgb,(inner_x[degree],inner_y[degree]),(x[degree],y[degree]),255,2)
            cv2.circle(backtorgb,(x[degree],y[degree]),30, (0,255 ,255), 5)
    
    end = time.time()
    print(end - start)
    print(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg',backtorgb)
    

0525_images/CCD-3\1.jpg
0.2882552146911621
.\Detect\Stop3\1_Detect.jpg
0525_images/CCD-3\10.jpg
0.2712748050689697
.\Detect\Stop3\10_Detect.jpg
0525_images/CCD-3\100.jpg
0.28324246406555176
.\Detect\Stop3\100_Detect.jpg
0525_images/CCD-3\101.jpg
0.29923248291015625
.\Detect\Stop3\101_Detect.jpg
0525_images/CCD-3\102.jpg
0.27828502655029297
.\Detect\Stop3\102_Detect.jpg
0525_images/CCD-3\103.jpg
0.29321742057800293
.\Detect\Stop3\103_Detect.jpg
0525_images/CCD-3\104.jpg
0.3610684871673584
.\Detect\Stop3\104_Detect.jpg
0525_images/CCD-3\105.jpg
0.2782552242279053
.\Detect\Stop3\105_Detect.jpg
0525_images/CCD-3\106.jpg
0.2752680778503418
.\Detect\Stop3\106_Detect.jpg
0525_images/CCD-3\107.jpg
0.2872316837310791
.\Detect\Stop3\107_Detect.jpg
0525_images/CCD-3\108.jpg
0.27230358123779297
.\Detect\Stop3\108_Detect.jpg
0525_images/CCD-3\109.jpg
0.2792534828186035
.\Detect\Stop3\109_Detect.jpg
0525_images/CCD-3\11.jpg
0.2892270088195801
.\Detect\Stop3\11_Detect.jpg
0525_images/CCD-3\110.jpg
0.

0.27728819847106934
.\Detect\Stop3\62_Detect.jpg
0525_images/CCD-3\63.jpg
0.27224087715148926
.\Detect\Stop3\63_Detect.jpg
0525_images/CCD-3\64.jpg
0.2682826519012451
.\Detect\Stop3\64_Detect.jpg
0525_images/CCD-3\65.jpg
0.27722883224487305
.\Detect\Stop3\65_Detect.jpg
0525_images/CCD-3\66.jpg
0.27130722999572754
.\Detect\Stop3\66_Detect.jpg
0525_images/CCD-3\67.jpg
0.26828646659851074
.\Detect\Stop3\67_Detect.jpg
0525_images/CCD-3\68.jpg
0.270247220993042
.\Detect\Stop3\68_Detect.jpg
0525_images/CCD-3\69.jpg
0.2692725658416748
.\Detect\Stop3\69_Detect.jpg
0525_images/CCD-3\7.jpg
0.270275354385376
.\Detect\Stop3\7_Detect.jpg
0525_images/CCD-3\70.jpg
0.26831555366516113
.\Detect\Stop3\70_Detect.jpg
0525_images/CCD-3\71.jpg
0.26629137992858887
.\Detect\Stop3\71_Detect.jpg
0525_images/CCD-3\72.jpg
0.27131199836730957
.\Detect\Stop3\72_Detect.jpg
0525_images/CCD-3\73.jpg
0.2692859172821045
.\Detect\Stop3\73_Detect.jpg
0525_images/CCD-3\74.jpg
0.2682778835296631
.\Detect\Stop3\74_Detect.jpg